# 0. Imports

In [1]:
import pandas as pd
import numpy as np
import scipy
from typing import Any, Callable
import helpers
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from zipfile import ZipFile

In [2]:
%%html
<style>
table {float:left}
</style>

In [3]:
#Settings for the plots 
sns.set_style("ticks")
colors= sns.color_palette('colorblind')
plt.rc('xtick', labelsize=12) 
plt.rc('ytick', labelsize=12) 
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=13)
plt.rcParams['ytick.major.size'] = 7
plt.rcParams['ytick.minor.size'] = 6

sns.set_style("darkgrid", {'axes.grid' : False, 'ytick.left': True, 'xtick.bottom': True})

In [4]:
X_train = pd.read_csv("project-data-interpolated/df_X_train.csv", index_col=0)
y_train = pd.read_csv("project-data-interpolated/df_y_train.csv", index_col=0)

X_val = pd.read_csv("project-data-interpolated/df_X_val.csv", index_col=0)
y_val = pd.read_csv("project-data-interpolated/df_y_val.csv", index_col=0)

X_test = pd.read_csv("project-data-interpolated/df_X_test.csv", index_col=0)

In [5]:
X_train.replace(['NO','YES'],[False,True],inplace=True)
X_val.replace(['NO','YES'],[False,True],inplace=True)
Sa_T_values = [float(x) for x in X_train.columns[:105]]

## 0.1. Where are corrupted values (-999)?

First, we study where the corrupted values of `-999` usually stand.  

In [6]:
X_train_corrupted_columns = X_train.columns[X_train.eq(-999).any(axis=0)==True].values
print(f'In X_train, corrupted values are in columns: {X_train_corrupted_columns}')

X_val_corrupted_columns = X_val.columns[X_val.eq(-999).any(axis=0)==True].values
print(f'In X_val, corrupted values are in columns: {X_val_corrupted_columns}')

X_test_corrupted_columns = X_test.columns[X_test.eq(-999).any(axis=0)==True].values
print(f'In X_test, corrupted values are in columns: {X_test_corrupted_columns}')

corrupted_columns = X_test.columns[X_test.eq(-999).any(axis=0)==True].values

In X_train, corrupted values are in columns: ['magnitude' 'distance_closest' 'lowest_usable_frequency']
In X_val, corrupted values are in columns: ['magnitude' 'distance_closest' 'lowest_usable_frequency']
In X_test, corrupted values are in columns: ['magnitude' 'distance_closest' 'lowest_usable_frequency']


We see that we only have corrupted values in the columns `magnitude`, `distance_closest` and `lowest_usable_frequency`.

How many corrupted values per column? 

In [7]:
print(f"There is {X_train['magnitude'].eq(-999).sum()} number of -999 in X_train, magnitude")
print(f"There is {X_train['distance_closest'].eq(-999).sum()} number of -999 in X_train, distance_closest")
print(f"There is {X_train['lowest_usable_frequency'].eq(-999).sum()} number of -999 in X_train, lowest_usable_frequency")
print(f"And there is in TOTAL: {X_train.eq(-999).any(axis=1).sum()} number of -999 in X_train")

There is 7955 number of -999 in X_train, magnitude
There is 8323 number of -999 in X_train, distance_closest
There is 7949 number of -999 in X_train, lowest_usable_frequency
And there is in TOTAL: 8340 number of -999 in X_train


This means that there are several rows where at least two of the three columns have corrupted values simultaneously

In [8]:
Id_train_magn = X_train.index[X_train['magnitude'].eq(-999)]
Id_train_dcl = X_train.index[X_train['distance_closest'].eq(-999)]
Id_train_lfreq = X_train.index[X_train['lowest_usable_frequency'].eq(-999)]

In [9]:
magn_inter_dcl = np.intersect1d(Id_train_magn, Id_train_dcl)
magn_inter_lfreq = np.intersect1d(Id_train_magn, Id_train_lfreq)
lfreq_inter_dcl = np.intersect1d(Id_train_dcl, Id_train_lfreq)
magn_inter_dcl_inter_lfreq = np.intersect1d(magn_inter_dcl,magn_inter_lfreq)
print(f'There is {np.round(len(magn_inter_dcl_inter_lfreq)/8340*100,0)}% of the rows that have the 3 columns corrupted at the same time')

There is 95.0% of the rows that have the 3 columns corrupted at the same time


In [10]:
id_corrupted = X_train.index[X_train.eq(-999).any(axis=1)==True].to_list()
id_healthy = X_train.index[X_train.eq(-999).any(axis=1)==False].to_list()

First we try to do regression on values where there was no NaNs. Then we do the same with a larger dataset where we include the interpolated Sa(T)s. 

# 1. Regression using SVM

## 1.1 Using only full rows

### 1.1.1 Define the dataset

In [1]:
import pandas as pd
import numpy as np
import scipy
from typing import Any, Callable
import helpers
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from zipfile import ZipFile

In [2]:
X_train = pd.read_csv("project-data-merged/df_X_train.csv", index_col=0)
y_train = pd.read_csv("project-data-merged/df_y_train.csv", index_col=0)

X_val = pd.read_csv("project-data-merged/df_X_val.csv", index_col=0)
y_val = pd.read_csv("project-data-merged/df_y_val.csv", index_col=0)

X_test = pd.read_csv("project-data-merged/df_X_test.csv", index_col=0)

In [3]:
X_train.replace(['NO','YES'],[False,True],inplace=True)
X_val.replace(['NO','YES'],[False,True],inplace=True)
Sa_T_values = [float(x) for x in X_train.columns[:105]]

In [4]:
full_rows_train = X_train.index[X_train.isnull().any(axis=1) == False].tolist()
full_rows_val = X_val.index[X_val.isnull().any(axis=1) == False].tolist()

In [5]:
#Keep only the rows that are empty
empty_rows = X_train.index.to_list()
for ele in sorted(full_rows_train, reverse = True):
    del empty_rows[ele]

In [6]:
X_data = pd.concat([X_train.loc[full_rows_train], X_val], ignore_index=True)

In [7]:
print(f"There is {X_data['magnitude'].eq(-999).sum()} number of -999 in X_data, magnitude")
print(f"There is {X_data['distance_closest'].eq(-999).sum()} number of -999 in X_data, distance_closest")
print(f"There is {X_data['lowest_usable_frequency'].eq(-999).sum()} number of -999 in X_data, lowest_usable_frequency")
print(f"And there is in TOTAL: {X_data.eq(-999).any(axis=1).sum()} number of -999 in X_data of size {X_data.shape[0]}")

There is 1933 number of -999 in X_data, magnitude
There is 1941 number of -999 in X_data, distance_closest
There is 1933 number of -999 in X_data, lowest_usable_frequency
And there is in TOTAL: 1941 number of -999 in X_data of size 5943


So approximately 6.7% of the rows in X_data have -999s. We shall only take the healty rows to train our model. 

In [8]:
corrupted_columns = X_data.columns[X_data.eq(-999).any(axis=0)==True].values

id_corrupted = X_data.index[X_data.eq(-999).any(axis=1)==True].to_list()
id_healthy = X_data.index[X_data.eq(-999).any(axis=1)==False].to_list()

In [9]:
X_data = X_data.iloc[id_healthy,:]

In [10]:
X_training, X_testing = helpers.create_train_test_dataset_for_regression(X_data,0.8,0.2)

In [11]:
X_testing_original, X_testing_missing = helpers.hide_values_999(X_testing, percentage = 0.067 )

In [12]:
sum_empty = len(X_testing_missing.index[X_testing_missing.isnull().any(axis=1)==True].to_list())/len(X_testing_missing)
print(f'In X_testing_missing, {np.round(sum_empty*100,3)}% of the rows have -999s.')

In X_testing_missing, 6.617% of the rows have -999s.


The training set is ready

### 1.1.2. Imports for sci-kit learn

In [13]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm

In [14]:
regr_magn = svm.SVR()
regr_magn.fit(X_training.drop(columns='magnitude'), X_training['magnitude'])

regr_dist = svm.SVR()
regr_dist.fit(X_training.drop(columns='distance_closest'), X_training['distance_closest'])

regr_freq = svm.SVR()
regr_freq.fit(X_training.drop(columns='lowest_usable_frequency'), X_training['lowest_usable_frequency'])

SVR()

In [15]:
pred_magn = regr_magn.predict(X_testing_original.drop(columns='magnitude'))
pred_dist = regr_dist.predict(X_testing_original.drop(columns='distance_closest'))
pred_freq = regr_freq.predict(X_testing_original.drop(columns='lowest_usable_frequency'))

In [16]:
score_magn = helpers.score_estimation(X_testing_original['magnitude'].to_numpy(),pred_magn)
print(f'The score for magnitude regression is : {score_magn}')

score_dist = helpers.score_estimation(X_testing_original['distance_closest'].to_numpy(),pred_dist)
print(f'The score for distance_closest regression is : {score_dist}')

score_freq = helpers.score_estimation(X_testing_original['lowest_usable_frequency'].to_numpy(),pred_freq)
print(f'The score for lowest_usable_frequency regression is : {score_freq}')

The score for magnitude regression is : 0.5469430664635353
The score for distance_closest regression is : 72.83983060546805
The score for lowest_usable_frequency regression is : 0.20735687721131105


## 1.2. Using interpolated dataset

### 1.2.1 Define the dataset

In [1]:
import pandas as pd
import numpy as np
import scipy
from typing import Any, Callable
import helpers
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from zipfile import ZipFile

In [2]:
X_train = pd.read_csv("project-data-interpolated/df_X_train.csv", index_col=0)
y_train = pd.read_csv("project-data-interpolated/df_y_train.csv", index_col=0)

X_val = pd.read_csv("project-data-interpolated/df_X_val.csv", index_col=0)
y_val = pd.read_csv("project-data-interpolated/df_y_val.csv", index_col=0)

X_test = pd.read_csv("project-data-interpolated/df_X_test.csv", index_col=0)

In [3]:
X_train.replace(['NO','YES'],[False,True],inplace=True)
X_val.replace(['NO','YES'],[False,True],inplace=True)
Sa_T_values = [float(x) for x in X_train.columns[:105]]

In [4]:
X_data = pd.concat([X_train, X_val], ignore_index=True)

In [5]:
print(f"There is {X_data['magnitude'].eq(-999).sum()} number of -999 in X_data, magnitude")
print(f"There is {X_data['distance_closest'].eq(-999).sum()} number of -999 in X_data, distance_closest")
print(f"There is {X_data['lowest_usable_frequency'].eq(-999).sum()} number of -999 in X_data, lowest_usable_frequency")
print(f"And there is in TOTAL: {X_data.eq(-999).any(axis=1).sum()} number of -999 in X_data of size {X_data.shape[0]}")

There is 8122 number of -999 in X_data, magnitude
There is 8490 number of -999 in X_data, distance_closest
There is 8116 number of -999 in X_data, lowest_usable_frequency
And there is in TOTAL: 8507 number of -999 in X_data of size 119648


So approximately 7.1% of the rows in X_data have -999s. We shall only take the healty rows to train our model. 

In [6]:
corrupted_columns = X_data.columns[X_data.eq(-999).any(axis=0)==True].values

id_corrupted = X_data.index[X_data.eq(-999).any(axis=1)==True].to_list()
id_healthy = X_data.index[X_data.eq(-999).any(axis=1)==False].to_list()

In [7]:
X_data = X_data.iloc[id_healthy,:]

In [8]:
print(f"And there is in TOTAL: {X_data.eq(-999).any(axis=1).sum()} number of -999 in X_data of size {X_data.shape[0]}")

And there is in TOTAL: 0 number of -999 in X_data of size 111141


Change the -999 into NaNs

In [9]:
frac_empty = len(id_corrupted)/len(X_data)
print(f'In X_data, {np.round(frac_empty*100,3)}% of the rows have -999s.')

In X_data, 7.654% of the rows have -999s.


In [10]:
X_training, X_testing = helpers.create_train_test_dataset_for_regression(X_data,0.8,0.2)

In [11]:
X_testing_original, X_testing_missing = helpers.hide_values_999(X_testing,percentage=frac_empty)

In [12]:
sum_empty = len(X_testing_missing.index[X_testing_missing.isnull().any(axis=1)==True].to_list())/len(X_testing_missing)
print(f'In X_testing_missing, {np.round(sum_empty*100,3)}% of the rows have -999s.')

In X_testing_missing, 7.405% of the rows have -999s.


The training set is ready

### 1.2.2. Imports for sci-kit learn

In [13]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm

In [14]:
regr_magn = svm.SVR()
regr_magn.fit(X_training.drop(columns=corrupted_columns), X_training['magnitude'])

SVR()

In [21]:
regr_dist = svm.SVR()
regr_dist.fit(X_training.drop(columns=corrupted_columns), X_training['distance_closest'])

SVR()

In [17]:
regr_freq = svm.SVR()
regr_freq.fit(X_training.drop(columns=corrupted_columns), X_training['lowest_usable_frequency'])

SVR()

In [15]:
pred_magn = regr_magn.predict(X_testing_original.drop(columns=corrupted_columns))

In [22]:
pred_dist = regr_dist.predict(X_testing_original.drop(columns=corrupted_columns))

In [18]:
pred_freq = regr_freq.predict(X_testing_original.drop(columns=corrupted_columns))

In [23]:
score_magn = helpers.score_estimation(X_testing_original['magnitude'].to_numpy(),pred_magn)
print(f'The score for magnitude regression is : {score_magn}')

score_dist = helpers.score_estimation(X_testing_original['distance_closest'].to_numpy(),pred_dist)
print(f'The score for distance_closest regression is : {score_dist}')

score_freq = helpers.score_estimation(X_testing_original['lowest_usable_frequency'].to_numpy(),pred_freq)
print(f'The score for lowest_usable_frequency regression is : {score_freq}')

The score for magnitude regression is : 0.5910170080728514
The score for distance_closest regression is : 126.86491554934966
The score for lowest_usable_frequency regression is : 0.17330662201485872


The regression works well for `magnitude` and `lowest_usable_frequency` features but poorly for `distance_closest`. Therefore this method should be used only to impute the corrupted data of `magnitude` and `lowest_usable_frequency`. The remaining column could be dropped.

## 1.3. Finding the corrupted values for `magnitude` and `lowest_usable_frequency`

### 1.3.1. Import and prep the data 

In [1]:
import pandas as pd
import numpy as np
import scipy
from typing import Any, Callable
import helpers
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from zipfile import ZipFile

In [25]:
X_train = pd.read_csv("project-data-interpolated/df_X_train.csv", index_col=0)
y_train = pd.read_csv("project-data-interpolated/df_y_train.csv", index_col=0)

X_val = pd.read_csv("project-data-interpolated/df_X_val.csv", index_col=0)
y_val = pd.read_csv("project-data-interpolated/df_y_val.csv", index_col=0)

X_test = pd.read_csv("project-data-interpolated/df_X_test.csv", index_col=0)

In [34]:
X_train.replace(['NO','YES'],[False,True],inplace=True)
X_val.replace(['NO','YES'],[False,True],inplace=True)
X_test.replace(['NO','YES'],[False,True],inplace=True)
Sa_T_values = [float(x) for x in X_train.columns[:105]]

#### 1.3.1.1. Prep the training data

In [4]:
X_data = pd.concat([X_train, X_val], ignore_index=True)

In [5]:
print(f"There is {X_data['magnitude'].eq(-999).sum()} number of -999 in X_data, magnitude")
print(f"There is {X_data['distance_closest'].eq(-999).sum()} number of -999 in X_data, distance_closest")
print(f"There is {X_data['lowest_usable_frequency'].eq(-999).sum()} number of -999 in X_data, lowest_usable_frequency")
print(f"And there is in TOTAL: {X_data.eq(-999).any(axis=1).sum()} number of -999 in X_data of size {X_data.shape[0]}")

There is 8122 number of -999 in X_data, magnitude
There is 8490 number of -999 in X_data, distance_closest
There is 8116 number of -999 in X_data, lowest_usable_frequency
And there is in TOTAL: 8507 number of -999 in X_data of size 119648


So approximately 7.1% of the rows in X_data have -999s. We shall only take the healty rows to train our model. 

In [26]:
corrupted_columns = X_data.columns[X_data.eq(-999).any(axis=0)==True].values

id_corrupted = X_data.index[X_data.eq(-999).any(axis=1)==True].to_list()
id_healthy = X_data.index[X_data.eq(-999).any(axis=1)==False].to_list()

In [7]:
X_data = X_data.iloc[id_healthy,:]

In [8]:
print(f"And there is in TOTAL: {X_data.eq(-999).any(axis=1).sum()} number of -999 in X_data of size {X_data.shape[0]}")

And there is in TOTAL: 0 number of -999 in X_data of size 111141


Change the -999 into NaNs

In [9]:
frac_empty = len(id_corrupted)/len(X_data)
print(f'In X_data, {np.round(frac_empty*100,3)}% of the rows have -999s.')

In X_data, 7.654% of the rows have -999s.


Training data is ready

#### 1.3.1.2. Prep the corrupted data

In [36]:
id_train_corrupted = X_train.index[X_train.eq(-999).any(axis=1)==True].to_list()
id_val_corrupted = X_val.index[X_val.eq(-999).any(axis=1)==True].to_list()
id_test_corrupted = X_test.index[X_test.eq(-999).any(axis=1)==True].to_list()

train_corrupted = X_train.loc[id_train_corrupted,:]
val_corrupted = X_val.loc[id_val_corrupted,:]
test_corrupted = X_test.loc[id_test_corrupted,:]

In [11]:
print(f"And there is in TOTAL: {X_train.eq(-999).any(axis=1).sum()} number of -999 in X_train of size {X_train.shape[0]}")
print(f"And there is in TOTAL: {X_val.eq(-999).any(axis=1).sum()} number of -999 in X_val of size {X_val.shape[0]}")
print(f"And there is in TOTAL: {X_test.eq(-999).any(axis=1).sum()} number of -999 in X_test of size {X_test.shape[0]}")

And there is in TOTAL: 0 number of -999 in X_train of size 117136
And there is in TOTAL: 0 number of -999 in X_val of size 2512
And there is in TOTAL: 0 number of -999 in X_test of size 48352


Corrupted data is ready

### 1.3.2. Train the SVR

In [12]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm

In [14]:
regr_magn = svm.SVR()
regr_magn.fit(X_data.drop(columns=corrupted_columns), X_data['magnitude'])

SVR()

In [15]:
regr_freq = svm.SVR()
regr_freq.fit(X_data.drop(columns=corrupted_columns), X_data['lowest_usable_frequency'])

SVR()

### 1.3.3. Predict the corrupted data

In [37]:
pred_train_magn = regr_magn.predict(train_corrupted.drop(columns=corrupted_columns))

In [38]:
pred_train_freq = regr_freq.predict(train_corrupted.drop(columns=corrupted_columns))

In [39]:
pred_val_magn = regr_magn.predict(val_corrupted.drop(columns=corrupted_columns))

In [40]:
pred_val_freq = regr_freq.predict(val_corrupted.drop(columns=corrupted_columns))

In [45]:
pred_test_magn = regr_magn.predict(test_corrupted.drop(columns=corrupted_columns))

In [46]:
pred_test_freq = regr_freq.predict(test_corrupted.drop(columns=corrupted_columns))

### 1.3.4. Export the corrected data

In [43]:
X_train.loc[id_train_corrupted,'magnitude']=pred_train_magn
X_train.loc[id_train_corrupted,'lowest_usable_frequency']=pred_train_freq

In [44]:
X_val.loc[id_val_corrupted,'magnitude']=pred_val_magn
X_val.loc[id_val_corrupted,'lowest_usable_frequency']=pred_val_freq

In [47]:
X_test.loc[id_test_corrupted,'magnitude']=pred_test_magn
X_test.loc[id_test_corrupted,'lowest_usable_frequency']=pred_test_freq

In [51]:
EXPORT = True

if EXPORT :
    X_train.to_csv("project-data-interpolated-corrected/df_X_train.csv")
    y_train.to_csv("project-data-interpolated-corrected/df_y_train.csv")

    X_val.to_csv("project-data-interpolated-corrected/df_X_val.csv")
    y_val.to_csv("project-data-interpolated-corrected/df_y_val.csv")

    X_test.to_csv("project-data-interpolated-corrected/df_X_test.csv")
    
    # create a ZipFile object
    zipObj = ZipFile('project-data-interpolated-corrected/data-interpolated-corrected.zip', 'w')

    # Add multiple files to the zip
    zipObj.write('project-data-interpolated-corrected/df_X_train.csv')
    zipObj.write('project-data-interpolated-corrected/df_y_train.csv')
    zipObj.write('project-data-interpolated-corrected/df_X_val.csv')
    zipObj.write('project-data-interpolated-corrected/df_y_val.csv')
    zipObj.write('project-data-interpolated-corrected/df_X_test.csv')

    # close the Zip File
    zipObj.close()